# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
output_data_file = "../output_data/cities.csv"

df = pd.read_csv(output_data_file)
df
vacation_data_df = df.drop(columns=["City_ID"])
vacation_data_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,puerto ayora,-0.74,-90.35,80.01,87,97,8.01,EC,1587606423
1,ushuaia,-54.80,-68.30,39.20,64,100,8.05,AR,1587606510
2,jamestown,42.10,-79.24,32.00,81,90,8.05,US,1587606425
3,busselton,-33.65,115.33,70.00,62,94,5.84,AU,1587606517
4,georgiyevskoye,58.73,45.02,25.56,92,100,11.25,RU,1587606517
...,...,...,...,...,...,...,...,...,...
552,aasiaat,68.71,-52.87,34.50,84,100,5.79,GL,1587606631
553,kedzierzyn-kozle,50.35,18.23,41.00,70,0,1.01,PL,1587606632
554,san jeronimo,-11.95,-75.28,50.50,29,98,1.83,PE,1587606632
555,kutum,14.20,24.67,62.15,16,97,4.45,SD,1587606632


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
locations = vacation_data_df[["Lat", "Lng"]]
humidity = vacation_data_df["Humidity"].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig




Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# A max temperature lower than 80 degrees but higher than 70
# Wind speed less than 10 mph
# Zero cloudiness
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal

temp = vacation_data_df[(vacation_data_df["Max Temp"] < 80) & (vacation_data_df["Max Temp"] > 70)]
wind = vacation_data_df[(vacation_data_df["Wind Speed"] < 10)]
Zero_cloud = vacation_data_df[(vacation_data_df["Cloudiness"]  == 0)]

new_df = pd.DataFrame(vacation_data_df({"City": city,
                                        "Lat": lat,
                                        "Lng": lon,
                                        "Max Temp": temp,
                                        "Humidity": humidity,
                                        "Cloudiness": Zero_cloud,
                                        "Wind Speed": wind,
                                        "Country": country,
                                        "Date": date}))
new_df

NameError: name 'city' is not defined

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = new_df
hotel_df["Hotel Name"] = ""


In [11]:
my_address = ""
params ={ "radius": 5000,
           "types": "lodging",
           "address": my_address,
           "key": g_key
        }
base_url = "https://maps.googleapis.com/maps/api/geocode/json"
my_geo = requests.get(base_url, params).json()
my_geo

lat = my_geo["results"][0]["geometry"]["location"]["lat"]
lng = my_geo["results"][0]["geometry"]["location"]["lng"]
target_search = "Hotel"

params = {
    "location": f"{lat},{lng}",
    "types": target_search,
    "rankby": "distance",
    "key": gkey
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

response = requests.get(base_url, params)
hotels_nearby = response.json()
try:
    print(hotels_nearby["results"][0]["name"])
    print(hotels_nearby["results"][0]["vicinity"])
except KeyError:
    print("Hotel not found")


{'error_message': "Invalid request. Missing the 'address', 'components', 'latlng' or 'place_id' parameter.",
 'results': [],
 'status': 'INVALID_REQUEST'}

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig



NameError: name 'rating' is not defined